In [1]:
import pandas as pd
import numpy as np

student_data = pd.read_csv("ML - Curricular Analytics - PIDM ONLY & Fixed Repeat IND.csv", low_memory=False)
grades = pd.read_csv("parsed_grades.csv")

In [2]:
student_data = student_data.drop(columns=['Admit_Term', 'Admit_Major_Code'])
student_data = student_data.dropna(subset=['Term', 'CRN', 'SUBJ', 'CRSE_NUMB', 'FINAL_GRADE'])

In [3]:
# student_data[(student_data["Pidm"] == 14523) & (student_data["Term"] == 201808)]

In [4]:
#Save latest student major
student_data['Term'] = student_data['Term'].astype(int)

last_sem_idx = student_data.groupby('Pidm')['Term'].idxmax()
latest_majors = student_data.loc[last_sem_idx, ['Pidm', 'Major_Desc']]
majors_dict = latest_majors.set_index('Pidm')['Major_Desc'].to_dict()
student_data['Lastest_Major'] = student_data['Pidm'].map(majors_dict)
student_data = student_data.drop(columns=['Major_Desc'])
student_data.drop_duplicates(subset=['Pidm', 'Term', 'CRN'], inplace=True)


In [5]:
#Add Numeric GPA Column

# Merge the two dataframes to bring in the Quality Points and whether to count in GPA
student_data = pd.merge(student_data, grades[['Code', 'Quality Points', 'Count in GPA?']], 
              left_on='FINAL_GRADE', right_on='Code', how='left', suffixes=('', '_grades'))

# Fill missing 'Quality Points' with 0.0 for unrecognized grades
student_data['Quality Points'] = student_data['Quality Points'].fillna(0.0)
student_data['Count_in_GPA'] = student_data['Count in GPA?'] == 'Y'

# student_data.shape
student_data.head()

,Pidm,Admit_Code,Admit_Desc,Admit_Level,Admit_College,Trump_Race,Trump_Race_Desc,MULTI,Race,NEW_ETHNICITY,...,CRN,SUBJ,CRSE_NUMB,REPEAT_IND,FINAL_GRADE,Lastest_Major,Code,Quality Points,Count in GPA?,Count_in_GPA
0,285,FS,Former USF Student Returning,UG,SA,W,White,NaN,W White,NaN,...,10803.0,LIS,3361,NaN,A,General Studies,A,4.00,Y,True
1,285,FS,Former USF Student Returning,UG,SA,W,White,NaN,W White,NaN,...,16614.0,LIS,3353,NaN,A-,General Studies,A-,3.67,Y,True
2,285,FS,Former USF Student Returning,UG,SA,W,White,NaN,W White,NaN,...,24972.0,COP,2030,NaN,B+,General Studies,B+,3.33,Y,True
3,285,FS,Former USF Student Returning,UG,SA,W,White,NaN,W White,NaN,...,26205.0,CIS,4510,NaN,A,General Studies,A,4.00,Y,True
4,285,FS,Former USF Student Returning,UG,SA,W,White,NaN,W White,NaN,...,54564.0,LIS,4365,NaN,W,General Studies,NaN,0.00,NaN,False


In [6]:
display(student_data[student_data["Pidm"] == 17439])

,Pidm,Admit_Code,Admit_Desc,Admit_Level,Admit_College,Trump_Race,Trump_Race_Desc,MULTI,Race,NEW_ETHNICITY,...,CRN,SUBJ,CRSE_NUMB,REPEAT_IND,FINAL_GRADE,Lastest_Major,Code,Quality Points,Count in GPA?,Count_in_GPA
451,17439,FS,Former USF Student Returning,UG,SA,H,Hispanic or Latino,NaN,W White,Hispanic or Latino,...,86919.0,ECH,3240L,NaN,C+,Engineering,C+,2.33,Y,True
452,17439,FS,Former USF Student Returning,UG,SA,H,Hispanic or Latino,NaN,W White,Hispanic or Latino,...,12263.0,BME,4406,NaN,C,Engineering,C,2.00,Y,True
453,17439,FS,Former USF Student Returning,UG,SA,H,Hispanic or Latino,NaN,W White,Hispanic or Latino,...,12266.0,ECH,4323C,NaN,D,Engineering,D,1.00,Y,True
454,17439,FS,Former USF Student Returning,UG,SA,H,Hispanic or Latino,NaN,W White,Hispanic or Latino,...,12268.0,ECH,4615,E,IF,Engineering,IF,0.00,Y,True
455,17439,FS,Former USF Student Returning,UG,SA,H,Hispanic or Latino,NaN,W White,Hispanic or Latino,...,20897.0,ECH,4241L,NaN,C+,Engineering,C+,2.33,Y,True
456,17439,FS,Former USF Student Returning,UG,SA,H,Hispanic or Latino,NaN,W White,Hispanic or Latino,...,11868.0,ECH,4615,I,B,Engineering,B,3.00,Y,True
457,17439,FS,Former USF Student Returning,UG,SA,H,Hispanic or Latino,NaN,W White,Hispanic or Latino,...,11906.0,BCH,6746,NaN,N,Engineering,NaN,0.00,NaN,False


In [7]:
#Final and Semester GPAs (Assuming all classes are equal credits)
student_data['Valid_Grades'] = np.where(student_data['Count in GPA?'] == 'Y' , student_data['Quality Points'], np.nan)
student_final_gpa = student_data.groupby('Pidm')['Valid_Grades'].mean().reset_index()
student_data = student_data.merge(student_final_gpa, on='Pidm', how='left', suffixes=('', '_mean'))
student_data.rename(columns={'Valid_Grades_mean':'Final GPA'}, inplace=True)

student_semester_gpa = student_data.groupby(['Pidm', 'Term'])['Valid_Grades'].mean().reset_index()
student_data = student_data.merge(student_semester_gpa, on=['Pidm', 'Term'], how='left', suffixes=('', '_mean'))
student_data.rename(columns={'Valid_Grades_mean':'Semester GPA'}, inplace=True)


display(student_data[student_data["Pidm"] == 17439])

,Pidm,Admit_Code,Admit_Desc,Admit_Level,Admit_College,Trump_Race,Trump_Race_Desc,MULTI,Race,NEW_ETHNICITY,...,REPEAT_IND,FINAL_GRADE,Lastest_Major,Code,Quality Points,Count in GPA?,Count_in_GPA,Valid_Grades,Final GPA,Semester GPA
451,17439,FS,Former USF Student Returning,UG,SA,H,Hispanic or Latino,NaN,W White,Hispanic or Latino,...,NaN,C+,Engineering,C+,2.33,Y,True,2.33,1.776667,2.3300
452,17439,FS,Former USF Student Returning,UG,SA,H,Hispanic or Latino,NaN,W White,Hispanic or Latino,...,NaN,C,Engineering,C,2.00,Y,True,2.00,1.776667,1.3325
453,17439,FS,Former USF Student Returning,UG,SA,H,Hispanic or Latino,NaN,W White,Hispanic or Latino,...,NaN,D,Engineering,D,1.00,Y,True,1.00,1.776667,1.3325
454,17439,FS,Former USF Student Returning,UG,SA,H,Hispanic or Latino,NaN,W White,Hispanic or Latino,...,E,IF,Engineering,IF,0.00,Y,True,0.00,1.776667,1.3325
455,17439,FS,Former USF Student Returning,UG,SA,H,Hispanic or Latino,NaN,W White,Hispanic or Latino,...,NaN,C+,Engineering,C+,2.33,Y,True,2.33,1.776667,1.3325
456,17439,FS,Former USF Student Returning,UG,SA,H,Hispanic or Latino,NaN,W White,Hispanic or Latino,...,I,B,Engineering,B,3.00,Y,True,3.00,1.776667,3.0000
457,17439,FS,Former USF Student Returning,UG,SA,H,Hispanic or Latino,NaN,W White,Hispanic or Latino,...,NaN,N,Engineering,NaN,0.00,NaN,False,NaN,1.776667,NaN


In [8]:
#Student Classes & Points per Semester (As an array of strings)
student_data['class'] = (student_data['SUBJ'] + student_data['CRSE_NUMB']).astype(str)
semester_classes = student_data.groupby(['Pidm', 'Term']).agg({
    'FINAL_GRADE': list,
    'Quality Points': list, 
    'class': list,
    'CRN': list,
}).reset_index()

#Drop unecessary columns
student_data.drop(['SUBJ', 'REPEAT_IND', 'FINAL_GRADE', 'class', 'Code', 'Count in GPA?', 'Count_in_GPA', 'Valid_Grades'], axis=1, inplace=True)
# semester_classes.head()
# student_data.shape
# display(student_data[student_data["Pidm"] == 17439])

In [9]:
# student_data.head()
# student_data.shape
# display(student_data[student_data["Pidm"] == 17439])

In [10]:
student_data.head()

,Pidm,Admit_Code,Admit_Desc,Admit_Level,Admit_College,Trump_Race,Trump_Race_Desc,MULTI,Race,NEW_ETHNICITY,...,SAT-ERW,SATM,SAT_TOTAL,Term,CRN,CRSE_NUMB,Lastest_Major,Quality Points,Final GPA,Semester GPA
0,285,FS,Former USF Student Returning,UG,SA,W,White,NaN,W White,NaN,...,-,-,-,202101,10803.0,3361,General Studies,4.00,3.75,3.75
1,285,FS,Former USF Student Returning,UG,SA,W,White,NaN,W White,NaN,...,-,-,-,202101,16614.0,3353,General Studies,3.67,3.75,3.75
2,285,FS,Former USF Student Returning,UG,SA,W,White,NaN,W White,NaN,...,-,-,-,202101,24972.0,2030,General Studies,3.33,3.75,3.75
3,285,FS,Former USF Student Returning,UG,SA,W,White,NaN,W White,NaN,...,-,-,-,202101,26205.0,4510,General Studies,4.00,3.75,3.75
4,285,FS,Former USF Student Returning,UG,SA,W,White,NaN,W White,NaN,...,-,-,-,202105,54564.0,4365,General Studies,0.00,3.75,NaN


In [11]:
#Remove repeated rows in the demographic columns
student_data = student_data.groupby(['Pidm', 'Term']).agg({ 
    'Admit_Level': 'first', 
    'Admit_College': 'first',
    'Lastest_Major': 'first',
    'Trump_Race': 'first', 
    'Trump_Race_Desc': 'first', 
    'MULTI': 'first', 
    'Race': 'first', 
    'NEW_ETHNICITY': 'first', 
    'GENDER_Code': 'first', 
    'GENDER': 'first', 
    'CITZ_IND': 'first', 
    'CITZ_CODE': 'first', 
    'CITZ_DESC': 'first', 
    'Final_GPA': 'first', 
    'ACTE': 'first', 
    'ACTM': 'first', 
    'ACTR': 'first', 
    'ACTS': 'first', 
    'EACT': 'first', 
    'SAT-ERW': 'first', 
    'SATM': 'first', 
    'SAT_TOTAL': 'first', 
    'Final GPA': 'first',
    'Semester GPA': 'first'
}).reset_index()

display(student_data[student_data["Pidm"] == 17439])


,Pidm,Term,Admit_Level,Admit_College,Lastest_Major,Trump_Race,Trump_Race_Desc,MULTI,Race,NEW_ETHNICITY,...,ACTE,ACTM,ACTR,ACTS,EACT,SAT-ERW,SATM,SAT_TOTAL,Final GPA,Semester GPA
209,17439,201608,UG,SA,Engineering,H,Hispanic or Latino,None,W White,Hispanic or Latino,...,-,-,-,-,-,-,-,-,1.776667,2.3300
210,17439,201701,UG,SA,Engineering,H,Hispanic or Latino,None,W White,Hispanic or Latino,...,-,-,-,-,-,-,-,-,1.776667,1.3325
211,17439,201901,UG,SA,Engineering,H,Hispanic or Latino,None,W White,Hispanic or Latino,...,-,-,-,-,-,-,-,-,1.776667,3.0000
212,17439,202001,UG,SA,Engineering,H,Hispanic or Latino,None,W White,Hispanic or Latino,...,-,-,-,-,-,-,-,-,1.776667,NaN


In [12]:
#Add Semester grades and gpa points to df
student_data = student_data.merge(semester_classes[['Pidm', 'Term', 'FINAL_GRADE', 'Quality Points', 'class', 'CRN']], on=['Pidm', 'Term'], how='left')
student_data.rename(columns={'Final_GPA':'HS GPA', 'Term':'Semester','FINAL_GRADE':'Semester Grades', 'Quality Points':'Semester Points', 'class':'Classes'}, inplace=True)

display(student_data[student_data["Pidm"] == 17439])

# student_data.head()
# student_data.shape

,Pidm,Semester,Admit_Level,Admit_College,Lastest_Major,Trump_Race,Trump_Race_Desc,MULTI,Race,NEW_ETHNICITY,...,EACT,SAT-ERW,SATM,SAT_TOTAL,Final GPA,Semester GPA,Semester Grades,Semester Points,Classes,CRN
209,17439,201608,UG,SA,Engineering,H,Hispanic or Latino,None,W White,Hispanic or Latino,...,-,-,-,-,1.776667,2.3300,[C+],[2.33],[ECH3240L],[86919.0]
210,17439,201701,UG,SA,Engineering,H,Hispanic or Latino,None,W White,Hispanic or Latino,...,-,-,-,-,1.776667,1.3325,"[C, D, IF, C+]","[2.0, 1.0, 0.0, 2.33]","[BME4406, ECH4323C, ECH4615, ECH4241L]","[12263.0, 12266.0, 12268.0, 20897.0]"
211,17439,201901,UG,SA,Engineering,H,Hispanic or Latino,None,W White,Hispanic or Latino,...,-,-,-,-,1.776667,3.0000,[B],[3.0],[ECH4615],[11868.0]
212,17439,202001,UG,SA,Engineering,H,Hispanic or Latino,None,W White,Hispanic or Latino,...,-,-,-,-,1.776667,NaN,[N],[0.0],[BCH6746],[11906.0]


In [13]:
###Code from Varma to convert SATs
# List of score columns
score_columns = ['ACTE', 'ACTM', 'ACTR', 'ACTS', 'EACT', 'SAT-ERW', 'SATM', 'SAT_TOTAL', 'HS GPA']

# Convert score columns to numeric
for col in score_columns:
    student_data[col] = pd.to_numeric(student_data[col], errors='coerce')

# ACT to SAT conversion table
act_to_sat_conversion = {
    36: 1590, 35: 1540, 34: 1500, 33: 1460, 32: 1430, 31: 1400,
    30: 1370, 29: 1340, 28: 1310, 27: 1280, 26: 1240, 25: 1210,
    24: 1180, 23: 1140, 22: 1110, 21: 1080, 20: 1040, 19: 1010,
    18: 970, 17: 930, 16: 890, 15: 850, 14: 800, 13: 760,
    12: 710, 11: 670, 10: 630, 9: 590
}

# Convert EACT to SAT
def convert_act_to_sat(eact_score):
    if pd.isna(eact_score):
        return np.nan
    return act_to_sat_conversion.get(int(eact_score), np.nan)

# Apply the conversion to EACT scores
student_data['Converted_SAT'] = student_data['SAT_TOTAL']

# Identify where SAT_TOTAL is missing but EACT is available
mask = student_data['SAT_TOTAL'].isna() & student_data['EACT'].notna()

# Apply conversion
student_data.loc[mask, 'Converted_SAT'] = student_data.loc[mask, 'EACT'].apply(convert_act_to_sat)

# Step 4: Handle Remaining Missing Values

# Drop rows where Converted_SAT or Final_GPA is still NaN
student_data = student_data.dropna(subset=['Converted_SAT', 'HS GPA'])

In [14]:
#Correct datatypes and output to csv
student_data = student_data.astype(str)
student_data['Pidm'] = student_data['Pidm'].astype(int)
student_data['Final GPA'] = student_data['Final GPA'].astype(float).round(2)
student_data['Semester GPA'] = student_data['Semester GPA'].astype(float).round(2)
student_data['Semester'] = student_data['Semester'].astype(float).astype(int).astype(str)
# student_data.to_csv("formatted_data.csv", index=False)

In [17]:
student_ids = student_data['Pidm'].unique()
print(student_ids)

[    794    1210    1833 ... 3984750 3984772 3988409]


In [20]:
from sklearn.model_selection import train_test_split

train, testing_data = train_test_split(student_ids, test_size=0.2, random_state=50)
dev, test = train_test_split(testing_data, test_size=0.5, random_state=50)

In [22]:
train_set = student_data[student_data['Pidm'].isin(train)]
dev_set = student_data[student_data['Pidm'].isin(dev)]
test_set = student_data[student_data['Pidm'].isin(test)]

In [26]:
student_data.to_csv("full_set.csv", index=False)
train_set.to_csv("train_set.csv", index=False)
dev_set.to_csv("dev_set.csv", index=False)
test_set.to_csv("test_set.csv", index=False)